In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import datetime
import pandas as pd

from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelBinarizer

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
print(tf.__version__)

In [ ]:
# import tensorflow as tf

# # Connect to TPU
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.TPUStrategy(tpu)
#     print("TPU initialized")
# except:
#     print("TPU not found")
# print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

In [ ]:
def plot_losses(history):
    plt.rcParams['figure.figsize'] = [20, 5]
    f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)

    ax1.set_title('Losses')
    ax1.set_xlabel('epoch')
    ax1.legend(loc="upper right")
    ax1.grid()
    ax1.plot(history['loss'], label='Training loss')
    ax1.plot(history['val_loss'], label='Validation loss')
    ax1.legend()

    ax2.set_title('Accuracy')
    ax2.set_xlabel('epoch')
    ax2.legend(loc="upper right")
    ax2.grid()
    ax2.plot(history['accuracy'], label='Training accuracy')
    ax2.plot(history['val_accuracy'], label='Validation accuracy')
    ax2.legend()

    plt.show()

In [ ]:
for folder in [d for d in os.listdir('/kaggle/input/animalprediction/mg-animal-prediction-25-26')]:
    print(folder)

## Analyze images and format

In [ ]:
data_dir = '/kaggle/input/animalprediction/mg-animal-prediction-25-26/train_images'

num_samples = 0
num_categories = 0
# For each category, we:
# - Count it and the number of images in it
# - Plot the first image and print its size
# Finally we print the number of images inside each group
for folder in [d for d in os.listdir(data_dir)]:
    n_images = sum(1 for _ in os.listdir(data_dir + '/' + folder))
    num_samples += n_images
    num_categories += 1
    img = cv2.imread(data_dir + '/' + folder + '/' + os.listdir(data_dir + '/' + folder)[0])

    
    print(folder , ':', end=' ')
    print(img.shape)
    print(f"{n_images = }")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()
print(f"{num_samples = }")
num_validation_samples = int(num_samples *0.2)
num_train_samples = num_samples - num_validation_samples
print(f"{num_validation_samples = }")
print(f"{num_train_samples = }")
print(f"{num_categories = }")

Each image has different size and shape, so we'll have to reshape them (`target_size` in `datagen.flow_from_directory`)

## Under and oversampling

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

def get_class_weights(generator):
    """
    Compute class weights for imbalanced datasets when using flow_from_directory.
    
    Args:
        generator: Keras ImageDataGenerator.flow_from_directory object (train_generator)
    
    Returns:
        dict mapping class index -> weight
    """
    class_indices = generator.class_indices
    classes = np.unique(generator.classes)
    
    class_weights = compute_class_weight(
        class_weight="balanced",
        classes=classes,
        y=generator.classes
    )
    
    return dict(zip(classes, class_weights))


In [ ]:
print(os.listdir('./models'))

## Define hyperparameters

In [ ]:
target_size = (128,128)
batch_size = 128

## For model save
family = "Fractal"
name = "main"
model_path = None

## Create Image Generators to provide images to the model
We include data augmentation by rotating, shifting, zooming and flipping to add more generalization.

In [ ]:
# We create a generator to enrich data
datagen = ImageDataGenerator(
    rescale=1./255.,
    validation_split=0.1,  # 10% for validation
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=2004
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=2004
)

In [ ]:
def big_classifier():
    input = Input(shape=(target_size[0], target_size[1], 3))
    x = Conv2D(32, 3, activation='relu')(input)
    x = Conv2D(64, 3, activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(128, 3, activation='relu')(x)
    x = Conv2D(256, 3, activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(256, 3, activation='relu')(x)
    x = Conv2D(512, 3, activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(512, 3, activation='relu')(x)
    x = Conv2D(1024, 3, activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Flatten()(x)
    x = Dense(512, activation="relu")(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation="relu")(x)
    
    output = Dense(num_categories, activation="softmax")(x)
    return Model(input, output)


def fractal_classifier():
    input = Input(shape=(target_size[0], target_size[1], 3))

    # First convolucional
    x = Conv2D(16, kernel_size=3, strides=1, padding='same', name='conv_initial')(inputs)
    x = BatchNormalization(name='batchnorm_initial')(x)
    x = MaxPooling2D(pool_size=2, strides=2, name='pool_initial')(LeakyReLU(name='relu_initial')(x))

    # Second convolucional
    x = Conv2D(64, kernel_size=3, strides=1, padding='same', name='conv1')(x)
    x = BatchNormalization(name='batchnorm1')(x)
    x = MaxPooling2D(pool_size=2, strides=2, name='pool1')(LeakyReLU(name='relu1')(x))

    # Third convolucional
    x = Conv2D(128, kernel_size=3, strides=1, padding='same', name='conv2')(x)
    x = BatchNormalization(name='batchnorm2')(x)
    x = MaxPooling2D(pool_size=2, strides=2, name='pool2')(LeakyReLU(name='relu2')(x))

    # Fourth convolucional 
    x = Conv2D(256, kernel_size=3, strides=1, padding='same', name='conv3')(x)
    x = BatchNormalization(name='batchnorm3')(x)
    x = MaxPooling2D(pool_size=2, strides=2, name='pool3')(LeakyReLU(name='relu3')(x))

    # Fully connected
    x = Flatten()(x)
    x = Dense(1024)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.2)(x)
    x = Dense(512)(x)
    x = LeakyReLU()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    return Model(inputs, outputs)

Not good.

Change the architecturen rethink it

Maybe you need to oversample some categories

In [ ]:
# Create Model
if not model_path:
    model = fractal_classifier()
# Load the model
else:
    model = keras.models.load_model(model_path)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Train

In [ ]:
history = model.fit(
train_generator,
steps_per_epoch = train_generator.samples // train_generator.batch_size,
epochs = 20,
validation_data = validation_generator,
validation_steps = validation_generator.samples // validation_generator.batch_size,
verbose = 1,
class_weight=get_class_weights(train_generator))

In [ ]:
plot_losses(history.history)

### Second training

In [ ]:
history = model.fit(
train_generator,
steps_per_epoch = train_generator.samples // train_generator.batch_size,
epochs = 20,
validation_data = validation_generator,
validation_steps = validation_generator.samples // validation_generator.batch_size,
verbose = 1,
class_weight=get_class_weights(train_generator))

In [ ]:
plot_losses(history.history)

In [ ]:
# Current timestamp
timestamp = datetime.datetime.now().strftime("%m_%d_%H:%M")
## Make sure everything saves correctly
os.makedirs("models", exist_ok=True)
try:
    family = family
except:
    family= "generic"
try:
    name = name
except:
    name= "model"
model_path = f"models/{family}_{name}_{timestamp}.keras"

# Save the model
model.save(model_path)

## Submit evaluation

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255.
).flow_from_directory(
    "/kaggle/input/animalprediction/mg-animal-prediction-25-26/test_images",
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False #To mantain order and being able to metric
)


In [ ]:
 model = keras.models.load_model("/kaggle/working/models/BigClassifier_best_09_29_09:53.keras")

In [ ]:
# # Get predictions for all test samples
# preds = model.predict(test_datagen)

# import pandas as pd
# import numpy as np

# # Nombres de archivo (orden correcto gracias a shuffle=False)
filenames = [os.path.basename(f) for f in test_datagen.filenames]  

# # Índices predichos
# pred_indices = np.argmax(preds, axis=1)

# Mapeo de índices a nombres de clase
class_labels = {v: k for k, v in train_generator.class_indices.items()}
pred_classes = [class_labels[i] for i in pred_indices]

# Construir DataFrame
submissions = pd.DataFrame({
    "id": filenames,
    "category": pred_classes
})
submissions.set_index('id',inplace=True)
print(submissions.head())


In [ ]:
family = "BigClassifier"
name = "best"

In [ ]:
os.makedirs("submissions", exist_ok=True)
timestamp = datetime.datetime.now().strftime("%m_%d_%H:%M")
try:
    family = family
except:
    family= "generic"
try:
    name = name
except:
    name= "model"
submission_path = f"submissions/{family}_{name}_{timestamp}.csv"
submissions.to_csv(submission_path)